In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar10
from tensorflow.keras import layers

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Load the data and split it between train and test sets

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [ ]:
path = '/content/base/resnetCIFAR10_final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

313/313 [==============================] - 5s 13ms/step - loss: 1.7435 - accuracy: 0.6738


[1.7434871196746826, 0.673799991607666]

In [ ]:
path = '/content/independant_1/resnetCIFAR10_final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

In [ ]:
path = '/content/independant_2/resnetCIFAR10_final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

In [ ]:
path = '/content/student_10/resnetCIFAR10final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

313/313 [==============================] - 9s 14ms/step - loss: 2.7655 - accuracy: 0.5653


[2.7654783725738525, 0.5652999877929688]

In [ ]:
path = '/content/student_50/resnetCIFAR10final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

313/313 [==============================] - 5s 12ms/step - loss: 2.2983 - accuracy: 0.6283


[2.2982568740844727, 0.6283000111579895]

In [ ]:
path = '/content/student_90/resnetCIFAR10final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

313/313 [==============================] - 5s 12ms/step - loss: 2.2339 - accuracy: 0.6113


[2.2339377403259277, 0.611299991607666]

In [ ]:
interpreter = tf.lite.Interpreter("/content/quantised_8_bit/8_bit.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_default = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_default = pd.concat([results_default, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

67.28


In [ ]:
interpreter = tf.lite.Interpreter("/content/q16/16_bit.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_default = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_default = pd.concat([results_default, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

67.39
